## Solutions

### Exercise 1

We let

- 0 represent “low”  
- 1 represent “high”  

In [14]:
p, q = 0.1, 0.2  # Prob of leaving low and high state respectively

Here’s a pure Python version of the function

In [15]:
def compute_series(n):
    x = np.empty(n, dtype=int)
    x[0] = 1  # Start in state 1
    U = np.random.uniform(0, 1, size=n)
    for t in range(1, n):
        current_x = x[t-1]
        if current_x == 0:
            x[t] = U[t] < p
        else:
            x[t] = U[t] > q
    return x

Let’s run this code and check that the fraction of time spent in the low
state is about 0.666

In [16]:
n = 100000
x = compute_series(n)
print(np.mean(x == 0))  # Fraction of time x is in state 0

0.67027


Now let’s time it

In [17]:
qe.util.tic()
compute_series(n)
qe.util.toc()

TOC: Elapsed: 0:00:0.08


0.08388733863830566

Next let’s implement a Numba version, which is easy

In [18]:
from numba import jit

compute_series_numba = jit(compute_series)

Let’s check we still get the right numbers

In [19]:
x = compute_series_numba(n)
print(np.mean(x == 0))

0.67007


Let’s see the time

In [20]:
qe.util.tic()
compute_series_numba(n)
qe.util.toc()

TOC: Elapsed: 0:00:0.00


0.0019526481628417969

This is a nice speed improvement for one line of code

Now let’s implement a Cython version

In [21]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [22]:
%%cython
import numpy as np
from numpy cimport int_t, float_t

def compute_series_cy(int n):
    # == Create NumPy arrays first == #
    x_np = np.empty(n, dtype=int)
    U_np = np.random.uniform(0, 1, size=n)
    # == Now create memoryviews of the arrays == #
    cdef int_t [:] x = x_np
    cdef float_t [:] U = U_np
    # == Other variable declarations == #
    cdef float p = 0.1
    cdef float q = 0.2
    cdef int t
    # == Main loop == #
    x[0] = 1
    for t in range(1, n):
        current_x = x[t-1]
        if current_x == 0:
            x[t] = U[t] < p
        else:
            x[t] = U[t] > q
    return np.asarray(x)

In [23]:
compute_series_cy(10)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [24]:
x = compute_series_cy(n)
print(np.mean(x == 0))

0.67129


In [25]:
qe.util.tic()
compute_series_cy(n)
qe.util.toc()

TOC: Elapsed: 0:00:0.00


0.00342559814453125

The Cython implementation is fast, but not as fast as Numba